<a href="https://colab.research.google.com/github/dookda/cmu_udfire_gee/blob/main/predict_hp_using_lstm_gee_colab.ipynb_ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api geemap folium tensorflow scikit-learn rasterio

import ee
import geemap
import folium
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Flatten, TimeDistributed, Reshape
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import rasterio
from rasterio.transform import from_origin

In [3]:
# เริ่มต้นใช้งาน GEE
ee.Authenticate()
try:
    ee.Initialize(project="ee-sakda-451407")
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project="ee-sakda-451407")

In [ ]:
# กำหนดพื้นที่ศึกษา (ตำบลสุเทพ)
roi = ee.Geometry.Rectangle([98.9, 18.7, 99.0, 18.8])

# กำหนดช่วงวันที่
start_date = '2020-01-01'
end_date = '2024-12-31'

# ดึงข้อมูล MODIS Hotspot (Thermal Anomalies)
modis = ee.ImageCollection('MODIS/061/MOD14A1') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# ดึงข้อมูล vegetation index
modis_veg = ee.ImageCollection('MODIS/061/MOD13A2') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# สร้างฟังก์ชันที่ปลอดภัยกว่าในการเพิ่มข้อมูล
def safe_add_date_info(image):
    date = ee.Date(image.get('system:time_start'))
    doy = date.getRelative('day', 'year')
    week = date.getRelative('week', 'year')
    return image.addBands(ee.Image.constant(doy).rename('doy').toInt()) \
               .addBands(ee.Image.constant(week).rename('week').toInt())

# เพิ่มข้อมูลวันและสัปดาห์
modis = modis.map(safe_add_date_info)
modis_veg = modis_veg.map(safe_add_date_info)

# ฟังก์ชันที่ปลอดภัยกว่าในการรวม collection
def safe_combine_collections(img):
    # กำหนด default values
    default_img = ee.Image(0).rename(['NDVI', 'EVI'])

    # ดึงข้อมูล vegetation ที่ใกล้เคียงวันที่ที่สุด
    veg_coll = modis_veg.filterDate(img.date(), img.date().advance(8, 'day'))

    # ใช้ภาพแรกที่มีอยู่ หรือใช้ default
    veg_img = ee.Algorithms.If(
        veg_coll.size().gt(0),
        veg_coll.first(),
        default_img
    )

    return img.addBands(ee.Image(veg_img).select(['NDVI', 'EVI']))

# รวมข้อมูลทั้งหมด
collection = modis.map(safe_combine_collections)

# ฟังก์ชันที่ปลอดภัยในการสร้าง weekly composites
def safe_create_weekly_composites(start_date, weeks):
    def create_week_image(week_num):
        week_num = ee.Number(week_num)
        start = start_date.advance(week_num, 'week')
        end = start.advance(1, 'week')

        # ข้อมูลสำหรับสัปดาห์นี้
        week_coll = collection.filterDate(start, end)

        # ตรวจสอบว่ามีข้อมูลหรือไม่
        has_data = week_coll.size().gt(0)

        # สร้าง default image
        default_image = ee.Image(0).rename('MaxFRP') \
            .addBands(ee.Image(0).rename('FireMask')) \
            .addBands(ee.Image(0).rename('NDVI')) \
            .addBands(ee.Image(0).rename('EVI')) \
            .addBands(ee.Image(week_num.mod(52)).rename('week').toInt()) \
            .addBands(ee.Image(week_num.mod(365)).rename('doy').toInt())

        # ใช้ค่าเฉลี่ยหากมีข้อมูล หรือใช้ default
        week_image = ee.Image(ee.Algorithms.If(
            has_data,
            week_coll.mean(),
            default_image
        ))

        return week_image.set('week_start', start.millis(), 'week_number', week_num)

    # สร้างลิสต์ของสัปดาห์
    week_list = ee.List.sequence(0, weeks-1)

    # สร้าง collection จากลิสต์
    return ee.ImageCollection(week_list.map(create_week_image))

start_date_ee = ee.Date(start_date)
total_weeks = 260  # 5 ปี * 52 สัปดาห์
weekly_collection = safe_create_weekly_composites(start_date_ee, total_weeks)

# ฟังก์ชันดึงข้อมูลเป็น array
def safe_image_to_array(image, region, scale):
    try:
        # ตรวจสอบว่า image มี band ข้อมูลหรือไม่
        band_names = image.bandNames()
        has_bands = band_names.size().gt(0)

        image_to_use = ee.Algorithms.If(
            has_bands,
            image,
            ee.Image(0).rename('dummy')  # ใช้ dummy image ถ้าไม่มี band
        )

        arr = ee.Image(image_to_use).sampleRectangle(region=region, defaultValue=0)
        return arr
    except Exception as e:
        print(f"Error: {e}")
        return None

# ดึงข้อมูลเป็น numpy array
scale = 1000  # 1km resolution
bands = ['MaxFRP', 'FireMask', 'NDVI', 'EVI', 'doy', 'week']

# สร้างข้อมูล training
training_data = []
labels = []

for week in range(total_weeks - 1):  # ลบ 1 เพราะเราจะใช้ทำนายสัปดาห์ถัดไป
    try:
        current_week_img = ee.Image(weekly_collection.filter(ee.Filter.eq('week_number', week)).first())
        next_week_img = ee.Image(weekly_collection.filter(ee.Filter.eq('week_number', week + 1)).first())

        # ดึงข้อมูล hotspot ของสัปดาห์ถัดไป (เป็น label)
        hotspot_next = next_week_img.select('MaxFRP').gt(0)  # 1 มี hotspot, 0 ไม่มี

        # ดึงข้อมูล features ของสัปดาห์ปัจจุบัน
        features = current_week_img.select(bands)

        # แปลงเป็น array
        features_arr = safe_image_to_array(features, roi, scale)
        hotspot_arr = safe_image_to_array(hotspot_next, roi, scale)

        if features_arr and hotspot_arr:
            # รับข้อมูล
            features_info = features_arr.getInfo()
            hotspot_info = hotspot_arr.getInfo()

            # ตรวจสอบว่า properties มีข้อมูล
            if 'properties' in features_info and 'properties' in hotspot_info:
                training_data.append(features_info)
                labels.append(hotspot_info)

    except Exception as e:
        print(f"Error processing week {week}: {e}")
        continue

# แปลงเป็น numpy array (ถ้ามีข้อมูล)
if training_data and labels:
    X = np.array(training_data)
    y = np.array(labels)

    # ปรับขนาดข้อมูลให้เหมาะสมกับแบบจำลอง
    def create_sequences(data, labels, time_steps=4):
        X_seq, y_seq = [], []
        for i in range(time_steps, len(data)):
            X_seq.append(data[i-time_steps:i])
            y_seq.append(labels[i])
        return np.array(X_seq), np.array(y_seq)

    time_steps = 4  # ใช้ข้อมูล 4 สัปดาห์ย้อนหลังเพื่อทำนายสัปดาห์ถัดไป
    X_seq, y_seq = create_sequences(X, y, time_steps)

    # แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
    split_ratio = 0.8
    split_idx = int(len(X_seq) * split_ratio)

    X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
    y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

    # สร้างแบบจำลอง CNN-LSTM
    n_timesteps, n_rows, n_cols, n_channels = X_train.shape[1], X_train.shape[2], X_train.shape[3], X_train.shape[4]

    model = Sequential()
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'),
                             input_shape=(n_timesteps, n_rows, n_cols, n_channels)))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(64))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    # ฝึกแบบจำลอง
    history = model.fit(X_train, y_train,
                       epochs=50,
                       batch_size=16,
                       validation_data=(X_test, y_test),
                       verbose=1)

    # ประเมินแบบจำลอง
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)

    # คำนวณ metrics
    accuracy = accuracy_score(y_test.flatten(), y_pred.flatten())
    precision = precision_score(y_test.flatten(), y_pred.flatten())
    recall = recall_score(y_test.flatten(), y_pred.flatten())
    f1 = f1_score(y_test.flatten(), y_pred.flatten())
    conf_matrix = confusion_matrix(y_test.flatten(), y_pred.flatten())

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    # พล็อตการเรียนรู้
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # ทำนาย hotspot สำหรับสัปดาห์ถัดไป
    latest_data = X[-time_steps:]
    latest_data = latest_data.reshape(1, time_steps, n_rows, n_cols, n_channels)

    prediction = model.predict(latest_data)
    hotspot_prediction = (prediction > 0.5).astype(int)

    # สร้าง raster จากผลการทำนาย
    def array_to_raster(array, bounds, crs='EPSG:4326'):
        height, width = array.shape
        xmin, ymin, xmax, ymax = bounds

        transform = from_origin(xmin, ymax, (xmax-xmin)/width, (ymax-ymin)/height)

        with rasterio.open('prediction.tif', 'w',
                           driver='GTiff',
                           height=height,
                           width=width,
                           count=1,
                           dtype=rasterio.uint8,
                           crs=crs,
                           transform=transform) as dst:
            dst.write(array.astype(rasterio.uint8), 1)

    # รับขอบเขตของพื้นที่
    roi_bounds = roi.bounds().getInfo()['coordinates'][0]
    x_coords = [coord[0] for coord in roi_bounds]
    y_coords = [coord[1] for coord in roi_bounds]
    bounds = [min(x_coords), min(y_coords), max(x_coords), max(y_coords)]

    # บันทึกผลการทำนายเป็น raster
    array_to_raster(hotspot_prediction[0, :, :, 0], bounds)

    # สร้างแผนที่ Folium
    m = folium.Map(location=[18.78, 98.95], zoom_start=12)

    # เพิ่มผลการทำนายลงบนแผนที่
    folium.raster_layers.ImageOverlay(
        image='prediction.tif',
        bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
        opacity=0.7,
        name='Hotspot Prediction'
    ).add_to(m)

    # เพิ่ม layer control
    folium.LayerControl().add_to(m)

    # บันทึกแผนที่
    m.save('hotspot_prediction_map.html')

    # แสดงแผนที่
    display(m)

    # บันทึกแบบจำลอง
    model.save('hotspot_cnn_lstm_model.h5')

else:
    print("No data available for training. Please check your data sources and region.")